In [ ]:
import os
import psycopg2
import requests
import json
from flask import Flask, jsonify, request
from langchain.chat_models import init_chat_model
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
llm = ChatGroq(
  model="moonshotai/kimi-k2-instruct-0905",
  groq_api_key=GROQ_API_KEY,
  temperature=0,
)
from langchain.chat_models import init_chat_model

import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")




In [2]:
def get_db_connection():
    conn = psycopg2.connect(
        host='localhost',
        dbname=os.getenv('POSTGRES_DB'),
        user=os.getenv('POSTGRES_USER'),
        password=os.getenv('POSTGRES_PASSWORD')
    )
    return conn

In [26]:
conn = get_db_connection()
cur = conn.cursor()
cur.execute("SELECT * FROM bookings;")
users = cur.fetchall()
cur.close()
conn.close()
users
# json.dumps(users)

[(1,
  'nexturn',
  'xyz',
  datetime.date(2025, 10, 10),
  datetime.date(2025, 10, 20),
  [[-1, -1], [9, 17], [9, 17], [9, 17], [9, 17], [-1, -1], [-1, -1]],
  100)]

In [30]:
from datetime import datetime 
date_object = datetime.strptime("20251010", '%Y%m%d').date()
date_object


datetime.date(2025, 10, 10)

In [59]:
from datetime import datetime

def slot_not_inside(slots1, slots2):
    for i in range(7):
        if not (slots2[i][0] <= slots1[i][0] <= slots1[i][1] <= slots2[i][1]):
            return True
    return False

def slot_overlap(slots1, slots2):
    for i in range(7):
        if (slots1[i][0] == -1 and slots1[i][1] == -1) or (slots2[i][0] == -1 and slots2[i][1] == -1):
            continue
        if (slots2[i][0] <= slots1[i][0] <= slots2[i][1]) or (slots2[i][0] <= slots1[i][1] <= slots2[i][1]):
            return True
    return False

def date_overlap(start_date1, end_date1, start_date2, end_date2):
    return start_date1 <= end_date2 and start_date2 <= end_date1

def conflict(StartDate: str, EndDate: str, HoursBooked: list, WorkerName: str) -> bool:
    """Function to check for conflicts in bookings"""
    for h in range(len(HoursBooked)):
        if len(HoursBooked[h]) == 0:
            HoursBooked[h] = [-1, -1]
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute("Select AvailableHours from availability where WorkerName = %s;", (WorkerName,))
    availability = cur.fetchone()[0]
    if slot_not_inside(HoursBooked, availability):
        return True
    cur.execute("SELECT StartDate, EndDate, HoursBooked from bookings where WorkerName = %s;", (WorkerName,))
    bookings = cur.fetchall()
    StartDate = datetime.strptime(StartDate, '%Y%m%d').date()
    EndDate = datetime.strptime(EndDate, '%Y%m%d').date()
    for b in bookings:
        if date_overlap(StartDate, EndDate, b[0], b[1]) and slot_overlap(HoursBooked, b[2]):
            print("Conflict with booking:", b, StartDate, EndDate, HoursBooked)
            return True
    return False


In [73]:
def get_data(table: str) -> str:
    """Return info of all users from given table"""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute("SELECT * FROM %s;" % (table,))
    content = cur.fetchall()
    cur.close()
    conn.close()
    if content is None:
        return 'No data found'
    return content

def add_user(UserName: str, UserPassword: str, UserType: str, Classification: str) -> str:
    """Function to add a user to the users table"""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute("INSERT INTO users (UserName, UserPassword, UserType, Classification) VALUES (%s, %s, %s, %s);", (UserName, UserPassword, UserType, Classification))
    conn.commit()
    cur.close()
    conn.close()
    return 'User added successfully'

def add_booking(BidderName: str, WorkerName: str, StartDate: str, EndDate: str, HoursBooked: list, HourlyRate: str) -> str:
    """Function to add a booking to the bookings table. Hours booked is a list of 7 lists, each containing start and end hours for each day of the week, beginning from Monday. for example, [[9,17], [], [], [], [], [10,16], []] means monday 9am to 5pm and and saturday 10 am to 4pm """
    for h in range(len(HoursBooked)):
        if len(HoursBooked[h]) == 0:
            HoursBooked[h] = [-1, -1]
    conn = get_db_connection()
    cur = conn.cursor()
    if conflict(StartDate, EndDate, HoursBooked, WorkerName):
        return 'Booking conflict detected'
    cur.execute("INSERT INTO bookings (BidderName, WorkerName, StartDate, EndDate, HoursBooked, HourlyRate) VALUES (%s, %s, %s, %s, %s, %s);", (BidderName, WorkerName, StartDate, EndDate, HoursBooked, HourlyRate))
    conn.commit()
    cur.close()
    conn.close()
    return 'Booking added successfully'

def add_availability(WorkerName: str, AvailableHours: list) -> str:
    """Function to add availability to the availability table when a worker type user adds their available hours. Available Hours is a list of 7 lists, each containing start and end hours for each day of the week, beginning from Monday. for example, [[9,17], [], [], [], [], [10,16], []] means monday 9am to 5pm and and saturday 10 am to 4pm """
    for h in range(len(AvailableHours)):
        if len(AvailableHours[h]) == 0:
            AvailableHours[h] = [-1, -1]
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute("INSERT INTO availability (WorkerName, AvailableHours) VALUES (%s, %s);", (WorkerName, AvailableHours))
    conn.commit()
    cur.close()
    conn.close()
    return 'Availability added successfully'

def check_availability(classification: str, StartDate: str, EndDate: str, HoursBooked: list) -> str:
    """Function to check availability of workers based on classification and booking hours. Hours booked is a list of 7 lists, each containing start and end hours for each day of the week, beginning from Monday. for example, [[9,17], [], [], [], [], [10,16], []] means monday 9am to 5pm and and saturday 10 am to 4pm """
    for h in range(len(HoursBooked)):
        if len(HoursBooked[h]) == 0:
            HoursBooked[h] = [-1, -1]
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute("SELECT UserName from users where usertype='worker' and classification = %s;", (classification,))
    workers = cur.fetchall()
    for w in workers:
        name = w[0]
        if conflict(StartDate, EndDate, HoursBooked, name):
            workers.remove(w)
    conn.commit()
    cur.close()
    conn.close()
    if len(workers) == 0:
        return 'No workers available'
    return workers

agent = create_react_agent(
    model=llm,
    tools=[get_data, add_user, add_booking, add_availability, check_availability],
    prompt="You are a postgres database management interface. If you can do multiple operations, do them all one by one, and return concise outputs."
)


In [63]:
# # Run the agent
# agent.invoke(
#     {"messages": [{"role": "user", "content": '''{
#         "operation": "add user",
#         "name": "def",
#         "password": "def123",
#         "type": "worker",
#         "AvailableHours": Monday, Wednesday and Fridays from 10am to 6pm,
#         "classification": "developer"
#     }'''}]
#     }
# )

In [64]:
# # Run the agent
# agent.invoke(
#     {"messages": [{"role": "user", "content": '''{
#         "operation": "check availability",
#         "StartDate": 20251010,
#         "EndDate": 20251020,
#         "HoursBooked": [[], [9,17], [9,17], [9,17], [9,17], [], []],
#         "classification": "developer"

#     }'''}]
#     }
# )

In [65]:
# add_availability("Alice", [[], [9, 17], [9, 17], [9, 17], [9, 17], [], []])

In [66]:
# agent.invoke(
#     {"messages": [{"role": "user", "content": "Get all data from the tables users"}]}
# )


In [67]:
# # Run the agent
# agent.invoke(
#     {"messages": [{"role": "user", "content": '''{
#         "operation": "add user",
#         "name": "nexturn",
#         "password": "nexturn123",
#         "type": "bidder",
#         "classification": "tech"
#     }'''}]
#     }
# )

In [68]:
# # Run the agent
# agent.invoke(
#     {"messages": [{"role": "user", "content": '''{
#         "operation": "add user",
#         "name": "xyz",
#         "password": "xyz123",
#         "type": "worker",
#         "AvailableHours": [[], [9,17], [9,17], [9,17], [9,17], [], []],
#         "classification": "developer"
#     }'''}]
#     }
# )

In [72]:
# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": '''{
        "operation": "add booking",
        "BidderName": "nexturn",
        "Workername": "abc",
        "StartDate": 20251021,
        "EndDate": 20251030,
        "HoursBooked": Tuesday to Friday from 10am to 3pm,
        "HourlyRate": 100
    }'''}]
    }
)

{'messages': [HumanMessage(content='{\n        "operation": "add booking",\n        "BidderName": "nexturn",\n        "Workername": "abc",\n        "StartDate": 20251021,\n        "EndDate": 20251030,\n        "HoursBooked": Tuesday to Friday from 10am to 3pm,\n        "HourlyRate": 100\n    }', additional_kwargs={}, response_metadata={}, id='baaa8ad8-9241-4ba9-acc1-0cc0dd25a1a2'),
  AIMessage(content="I'll help you add this booking to the database. First, let me convert the hours format to the required structure, then add the booking.", additional_kwargs={'tool_calls': [{'id': 'functions.add_booking:0', 'function': {'arguments': '{"BidderName":"nexturn","EndDate":"20251030","HourlyRate":"100","HoursBooked":[[],["10:00","15:00"],["10:00","15:00"],["10:00","15:00"],["10:00","15:00"],[],[]],"StartDate":"20251021","WorkerName":"abc"}', 'name': 'add_booking'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 132, 'prompt_tokens': 708, 'total_tokens': 840, 'com

In [56]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": '''{
        "operation": "get",
        "table": "users"
    }'''}]
    }
)
print(response["messages"][-1].content)

Here are all users in the `users` table:

| UserID | UserName | Password   | UserType | Classification |
|--------|----------|------------|----------|------------------|
| 1      | nexturn  | nexturn123 | bidder   | tech             |
| 2      | xyz      | xyz123     | worker   | developer        |


In [147]:
from datetime import datetime, timedelta

def previous_monday(date_int):
    date = datetime.strptime(str(date_int), "%Y%m%d")
    days_to_subtract = (date.weekday())
    previous_monday_date = date - timedelta(days=days_to_subtract)
    return int(previous_monday_date.strftime("%Y%m%d"))

def expand_slots(slots):
    result = []
    for start, end in slots:
        result.extend(range(start, end + 1))
    return result

def expand_slots_duration(slots, basedate, start_date, duration):
    basedate_date = datetime.strptime(str(basedate), "%Y%m%d")
    start_date_date = datetime.strptime(str(start_date), "%Y%m%d")
    end_date_date = datetime.strptime(str(basedate), "%Y%m%d") + timedelta(weeks=duration)
    diff_days = (start_date_date - basedate_date).days*24
    days = end_date_date - start_date_date
    result = []
    for i in range(((end_date_date - basedate_date).days + 1)*24):
        if i >= diff_days:
            for start, end in slots:
                if start <= i % 168 <= end:
                    result.append(i)
    return result


def has_overlap(slots1, slots2):
    return not set(slots1).isdisjoint(set(slots2))

def check_booking_overlap(search_slots, search_start_date, search_duration, booking_slots, booking_start_date, booking_duration):
    basedate = previous_monday(min(search_start_date, booking_start_date))
    search_slots_expanded = expand_slots_duration(search_slots, basedate, search_start_date, search_duration)
    booking_slots_expanded = expand_slots_duration(booking_slots, basedate, booking_start_date, booking_duration)
    return has_overlap(search_slots_expanded, booking_slots_expanded)

def check_admin_overlap(search_slots, admin_slots):
    return has_overlap(expand_slots(search_slots), expand_slots(admin_slots))

In [161]:
def search_booking(Position: str, Slots: list, StartDate: str, Duration: int) -> str:
    """Function to search for bookings based on position and slots"""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute("SELECT WorkerName from workers where Position = %s;", (Position,))
    worker = cur.fetchall()
    worker_names = [w[0] for w in worker]
    cur.execute("Select WorkerName, slots from bookings where WorkerName = ANY(%s) and BidderName = 'admin';", (worker_names,))
    slots = cur.fetchall()
    for s in slots:
        if check_admin_overlap(Slots, s[1]):
            worker_names.remove(s[0])
    cur.execute("Select WorkerName, slots, startdate, duration from bookings where WorkerName = ANY(%s) and BidderName != 'admin';", (worker_names,))
    slots = cur.fetchall()
    for s in slots:
        search_slots = Slots
        search_start_date = int(StartDate)
        search_duration = Duration
        booking_slots = s[1]
        booking_start_date = int(s[2].strftime('%Y%m%d'))
        booking_duration = s[3]
        if check_booking_overlap(search_slots, search_start_date, search_duration, booking_slots, booking_start_date, booking_duration):
            worker_names.remove(s[0])
    # print("Available workers:", worker_names)
    cur.close()
    conn.close()
    return json.dumps(worker_names)

search_booking("developer", [[10, 18]], "20250101", 10)

'[]'